# Использование ArcGIS API for Python в Jupiter Notebook

## Введение
Все чаще и чаще у аналитиков данных (или как еще их называют — Data Scientist) появляется потребность в визуализации данных на карте. Какой инструмент сейчас считается наиболее удобным для работы аналитика? Конечно же, тетрадки! До последнего времени возможностей по визуализации геоданных было не так много. Можно было делать статические растры в matplotlib, иногда можно было добавлять даже базовые карты. Интересной оказалась библиотека для работы с Leaflet, где можно открывать geojson-файлы. Сегодня же я хочу рассказать об ArcGIS API for Python от компании Esri.

Эта статья будет полезна как аналитикам, желающим изучить примеры работы с ГИС, так и картографам и ГИС-специалистам, которым интересно попробовать себя в написании кода.

## Что такое ArcGIS API for Python?
Это простая в использовании и богатая по функционалу библиотека для визуализации и анализа геоданных, а также для управления корпоративной ГИС (Географический Информационной Системы), от пространственных данных до администрирования. Важной особенностью этой библиотеки является "питоничность" (Pythonic) с использованием стандартных конструкций, структур и идиом, что выгодно отличает ее от стандартной ArcGIS-библиотеки arcpy, которая порой вызывала ярость у разработчиков как стилем кода, так и быстродействием.

В этой статье пойдет речь о тех функциях, которые доступны без покупки лицензий ArcGIS или подписки ArcGIS Online. Для работы понадобится бесплатный аккаунт ArcGIS Online (я опущу подробности его создания - это довольно просто сделать на http://www.arcgis.com). В этом варианте присутствует ограничение на 1000 объектов в слое.

API распространяется в виде conda-пакета arcgis. Внутри пакета, который представляет собой концептуальную модель ГИС, функциональность организована в нескольких различных модулей, что делает его простым в использовании и понимании. Каждый модуль имеет несколько типов и функций, которые охватывают конкретный аспект ГИС.
В этой статье я рассмотрю небольшую часть API в качестве вводного материала.
* `arcgis.gis` обеспечивает базовую информационную модель как для корпоративной ГИС, так и для ArcGIS Online. Этот модуль обеспечивает функции управления (создание, чтение, обновление и удаление) пользователями, группами и контентом. Собственно, это ваша точка входа в ГИС.
* `arcgis.widgets` - это, собственно, модуль для управления картографическими виджетами (MapView) в Jupyter notebook.

## Установка
На данный момент библиотека распространяется только внутри Anaconda for Python 3x. Ставится все просто.

Надеюсь, все прошло успешно. Теперь можно начинать использование.
Если нет желания ставить Anaconda, но библиотеку протестировать хочется, то можно попробовать песочницу от Esri: https://notebooks.esri.com.

## Использование
Объект GIS в модуле gis - самый главный в ArcGIS API for Python, ваша входная точка. Он олицетворяет ту ГИС, с которой вы работаете, будь ArcGIS Online или корпоративный ArcGIS. Объект позволяет работать с геоконтентом или же администрировать ГИС. Начнем с инициализации объекта GIS

In [1]:
import arcgis
from arcgis.gis import GIS

Для работы можно использовать как базовое анонимное подключение к ArcGIS Online...

In [2]:
gis = GIS()

...так и с логином.

In [ ]:
my_gis = GIS(url='http://arcgis.com', username='andrey_zhukov', password='секрет')

Для любого инструмента можно вызывать справку, например

In [4]:
gis?

Кроме того, поддерживается и контекстный ввод

<img src='http://esri.github.io/arcgis-python-api/notebooks/nbimages/01-dropdown.png'>

Встроить карту - это просто!
Давайте инициализируем объект карты hello_map с центром в Москве и с уровнем зума 15.

In [5]:
hello_map = my_gis.map("Москва", zoomlevel = 15)

А теперь вызовем его...

In [6]:
hello_map

Добро пожаловать!
В карте работают базовые элементы - перетаскивание мышью (pan) и изменение масштаба (zoom) как колесом мыши (или соответствующим жестом на тачпаде), так и экранными кнопками.
Последующие манипуляции с объетом hello_map отразятся во фрейме выше.
Чем можно управлять у карты?
Например, зумом.

In [161]:
hello_map.zoom = 10

Увеличим площадь карты.

In [7]:
hello_map.height = '600px'

Посмотрим список доступных базовых карт.

In [162]:
hello_map.basemaps

['streets',
 'satellite',
 'hybrid',
 'topo',
 'gray',
 'dark-gray',
 'oceans',
 'national-geographic',
 'terrain',
 'osm']

Можно все посмотреть? Конечно!

In [163]:
from time import sleep
for basemap in hello_map.basemaps:
    hello_map.basemap = basemap
    sleep(5)

OpenStreetMap - вполне привычно, остановимся на нем.

In [15]:
hello_map.basemap = 'osm'

Поищем адреса. Найдем адрес БЦ Омега плаза и приблизм карту к нему.
Геокодирование по одному адресу абсолютно бесплатно, но пакетное геокодирование уже требует кредитов на счете организации.

In [8]:
location = arcgis.geocoding.geocode('Москва, Ленинская слобода, 19', max_locations=1)[0]

In [9]:
hello_map.extent = location['extent']

На карту можно добавлять контент из галереи ArcGIS Online. Поищем что-нибудь интересное...

In [11]:
from IPython.display import display
items = gis.content.search('title:Moscow', item_type='Feature Collection')
for item in items:
    display(item)

<Item title:"Moscow" type:Feature Collection owner:southside_institute>

<Item title:"Moscow Folklore Map" type:Feature Collection owner:nivirov>

<Item title:"Moscow Walking Tour" type:Feature Collection owner:thomaskoblet>

<Item title:"Moscow_route" type:Feature Collection owner:OSerebryannaya_i18n>

<Item title:"Moscow_route_rechts" type:Feature Collection owner:OSerebryannaya_i18n>

`Moscow Walking Tour` - отлично! Уточним элемент.

In [12]:
my_layer = items[2]
display(my_layer)

<Item title:"Moscow Walking Tour" type:Feature Collection owner:thomaskoblet>

Добавляем сервис и смотрим на наш hello_map

In [13]:
hello_map.add_layer(my_layer)

In [14]:
hello_map.zoom = 12

Мы получили веб-карту, встроенную во фрейм внутри Notebook. В ней поддерживается навигация и вывод информации по клику на объектах. На сегодняшний день многие организации используют технологии ArcGIS не только на корпоративном уровне, но и для публикации открытых данных. Некоторые из этих открытых данных доступны для поиска и использования на ArcGIS Online, другие же можно найти через портал открытых данных (https://opendata.arcgis.com/) и подгрузить в ArcGIS Online.

Happy ~~hacking~~ mapping!

## Работа с объектами
Конечно же, каждый Data Scientist, работающий в Jupiter Notebook, привык к pandas и DataFrame. Давайте поработаем уже с полноценным набором данных, используя pandas и ArcGIS.

Для этого сначала импортируем csv в pandas, а потом преобразуем его в слой карты. 
Как пример я возьму слой паркоматов Москвы. Я уже немного поколдовал с исходным csv. Перекодировал в utf8 и переименовал столбцы, чтобы не возникло неожиданных проблем.

In [16]:
import pandas as pd
parkomats = pd.read_csv(r'.\data\data-417-2017-02-14.csv', sep=';')

Посмотрим получившийся объект. В нем уже есть широта и долгота, а также псевдо-geojson-поле geoData. При импорте данных ArcGIS сам пытается опознать поля геометрии или адреса. Если бы нам требовалось собрать данные по адресам, то пришлось бы собирать адрес из нескольких полей.

In [17]:
parkomats.tail()

,ID,NumberOfParkingMeter,global_id,AdmArea,District,StreetName,Location,longitude,latitude,ParkingZoneNumber,geoData,Unnamed: 11
870,1990,Паркомат № 3287,282752273,Юго-Восточный административный округ,район Люблино,Совхозная улица,"Совхозная улица, дом 45",37.761807,55.675818,4080,"{type=Point, coordinates=[37.7618068305579, 55...",NaN
871,1991,Паркомат № 3397,282752274,Восточный административный округ,район Сокольники,5-я Сокольническая улица,"5-я Сокольническая улица, дом 1",37.688124,55.785373,4025,"{type=Point, coordinates=[37.6881239308419, 55...",NaN
872,1972,Паркомат № 2240,282752301,Западный административный округ,район Фили-Давыдково,улица Герасима Курина,"улица Герасима Курина, дом 16",37.472284,55.731046,4069,"{type=Point, coordinates=[37.4722838315654, 55...",NaN
873,1980,Паркомат № 2345,282752302,Западный административный округ,район Раменки,Университетский проспект,"Университетский проспект, дом 13",37.543410,55.701725,4068,"{type=Point, coordinates=[37.543409630392, 55....",NaN
874,1963,Паркомат № 2031,282920545,Северо-Западный административный округ,район Южное Тушино,Сходненская улица,"Сходненская улица, дом 50",37.440234,55.847444,4071,"{type=Point, coordinates=[37.4402341318386, 55...",NaN


Теперь импортирум DataFrame из pandas в объекты gis. Для этого просто вызываем функцию import_data. В результате у нас получается объект FeatureCollection

In [19]:
geoparkomats = my_gis.content.import_data(parkomats)
geoparkomats

<FeatureCollection>

Теперь добавим объекты на карту. Для этого инициализируем новый объект map и в него добавил наш новый слой.

In [20]:
parkomat_map = my_gis.map('Москва', zoomlevel=12)
parkomat_map.basemap = 'osm'
parkomat_map.add_layer(geoparkomats)
parkomat_map.height = '600px'
parkomat_map

Уже неплохо! Но неплохо было бы эти данные оформить. Давайте попробуем их разукрасить. Для начала просто по зонам парковки

In [21]:
parkomat_cat_map = my_gis.map('Москва', zoomlevel=10)
parkomat_cat_map.basemap = 'gray'
parkomat_cat_map.add_layer(geoparkomats, {"renderer":"ClassedColorRenderer", "field_name":"ParkingZoneNumber"})
parkomat_cat_map.height = '600px'
parkomat_cat_map

Многие любят теплокарты. Давайте попробуем сделать и ее.

In [22]:
parkomat_heat_map = my_gis.map('Москва', zoomlevel=11)
parkomat_heat_map.basemap = 'dark-gray'
parkomat_heat_map.add_layer(geoparkomats, {"renderer":"HeatmapRenderer"})
parkomat_heat_map.height = '600px'
parkomat_heat_map

Как видите, добавить и стилизовать слой - это не сложно.

Кроме того, можно расположить все карты рядом. Для этого создадим и настроим три новых фрейма и впишем их в один бокс.

In [23]:
just_map = my_gis.map('Москва', zoomlevel=10)
just_map.basemap = 'osm'
just_map.add_layer(geoparkomats)

In [24]:
cat_map = my_gis.map('Москва', zoomlevel=10)
cat_map.basemap = 'gray'
cat_map.add_layer(geoparkomats, {"renderer":"ClassedColorRenderer", "field_name":"ParkingZoneNumber"})

In [25]:
heat_map = my_gis.map('Москва', zoomlevel=10)
heat_map.basemap = 'dark-gray'
heat_map.add_layer(geoparkomats, {"renderer":"HeatmapRenderer"})

In [26]:
from ipywidgets import *
just_map.layout=Layout(flex='1 1', padding='5px')
cat_map.layout=Layout(flex='1 1', padding='5px')
heat_map.layout=Layout(flex='1 1', padding='5px')
box = HBox([just_map, cat_map, heat_map])
box.height = '300px'
box

## Заключение

Итак, вы узнали, как легко и просто использовать в Jupyter Notebook полноценные веб-карты с помощью ArcGIS API for Python. Это новый продукт, едва вышедший из беты, но уже довольно интересный, с большими перспективами.

Материалы этой статьи выложены на GitHub (https://github.com/fall-out-bug/arcgis_python). Код библиотеки и обширные материалы Esri тоже можно найти на GitHub (https://github.com/Esri/arcgis-python-api)

**Happy mapping!**